In [1]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
include("9msctrl.jl")
include("Initial condition variation.jl")
include("Perturbation time.jl")
include("Contour Plots.jl")
include("Magnitude Rand.jl")
include("Objective function Monitor.jl")
include("AdjointControl.jl")
include("RandomInitial.jl")
import LinearAlgebra: norm
using PyPlot; pygui(true)
using PyCall
np = pyimport("numpy")
sc = pyimport("scipy")
plt = pyimport("matplotlib.pyplot");

┌ Info: Recompiling stale cache file C:\Users\Kaijie\.julia\compiled\v1.2\ToySystems\chsON.ji for ToySystems [53c88820-6eff-5a8c-9a74-badd815e7be1]
└ @ Base loading.jl:1240
┌ Warning: Package ToySystems does not have Flows in its dependencies:
│ - If you have ToySystems checked out for development and have
│   added Flows as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ToySystems
└ Loading Flows into ToySystems from project dependency, future warnings for ToySystems are suppressed.


In [2]:
## Lyapunov exponent analysis without control
# Re = 350 #`transition Reynold number
# Initial = rand(9)
# Initialdel = Initial+mgscale(rand(9),1); # Difference in length of 0.01
# norm(Initialdel - Initial)

In [3]:
function rescaleinitial(u, delu, scale)
    vector = delu-u
    delu = u + [scale/norm(vector)] .*vector
    return delu
end

function print_factors(x)
   factors = []
   for i in 1:x
        if x % i == 0
            factors = push!(factors, i)
        else
            0 == 0
        end
    end
    return factors
end

function twooutput(u0, delu0, scale, Re, tstart, timestep, tend, tsplit)
    f = NineModeSystem(Re)
    tstart! = tstart
    tend! = tstart + tsplit
    n = (tend-tstart)/tsplit
#     outputu0 = [u0]
    outputd = []
#     outputdelu0 = [delu0]
    outputd1 = []
    u0 = copy(u0)
    delu0 = copy(delu0)
    for i in 1:n
        outputd1 = push!(outputd1, norm(delu0 - u0))
        ϕ = flow(f, RK4(zeros(9)), TimeStepConstant(timestep))
        u0plus = ϕ(u0,(tstart!,tend!))
        delu0PLUS = ϕ(delu0,(tstart!,tend!))
        outputd = push!(outputd, norm(delu0PLUS-u0plus))
        delu0plus = rescaleinitial(u0plus, delu0PLUS, scale)
#         outputu0 = push!(outputu0, u0plus)
#         outputdelu0 = push!(outputdelu0, delu0plus)
        u0 = u0plus
        delu0 = delu0plus
        tend! = tend! + tsplit
        tstart! = tend! - tsplit
    end
    λ = (1/tsplit) .* log.(outputd ./ outputd1)
    return sum(λ)/length(λ)
end

function plotlyapunov(number, Re::AbstractVector, Inidiff, Itediff, tstart, timestep, tend, energyradius = false, ISRANGE = false, d = zeros(9))
    u0_array = Inipointnum(number, energyradius, ISRANGE, d)
    delu0_array = []
    
    for i in 1:length(u0_array)
        delu0_array = push!(delu0_array,mgscale(rand(9),Inidiff) + u0_array[i])
    end
    checkpoint1 = norm.(delu0_array - u0_array)
    
    tgap = tend - tstart
    avaitsplit = print_factors(tgap)
    Y_ARRAY = []
    
    for i in 1:length(Re)
        Rey = Re[i]
        y_array = zeros(length(avaitsplit))
        for j in 1:length(avaitsplit)
            tsplit = avaitsplit[j]
            λaverage = 0
            for k in 1:number
                λaverage += twooutput(u0_array[k], delu0_array[k], Itediff, Rey, tstart, timestep, tend, tsplit)
            end
            y_array[j] = λaverage/number
        end
        Y_ARRAY = push!(Y_ARRAY, y_array)
    end
    return avaitsplit, Y_ARRAY
end

plotlyapunov (generic function with 4 methods)

In [4]:
tgroup = []
z = 0.1:0.1:1.0
for i in 1:length(0.1:0.1:1.0)
    tgroup = push!(tgroup, plotlyapunov(1000, 300:50:500, z[i], z[i], 0, 0.5, 10000, 0.3, true, [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))
    println("done")
end

done
done
done
done
done
done
done
done
done
done


In [5]:
fig, ax = plt.subplots()
t = 1
for i in 1:length(t2)
    ax.plot(t1, t2[i], label = "$t")
    t+=1
end

ax.legend()
xlabel(L"timeplit")
ylabel(L"average  λ")

PyObject Text(4.847222222222216, 0.5, '$average  λ$')

In [199]:
u_a = Inipointnum(10, 0.3, false, [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

10-element Array{Any,1}:
 [1.129163261832137, 0.05272474494245278, 0.06454826837452458, 0.1093113908748335, 0.07366004668106209, 0.0913378637284798, 0.041488269372518906, 0.14041813232540176, 0.13861658533406637]    
 [1.115455498332982, 0.11950398533759235, 0.03485068288013253, 0.10102653241133892, 0.146555272444867, 0.10290531922987346, 0.013189303704446527, 0.05123005041255408, 0.12689155464479154]    
 [1.1154545284899855, 0.0641064747069359, 0.12685508873076196, 0.0326533922442464, 0.006592277705277569, 0.14249179597663744, 0.09976664803913822, 0.14181545300012677, 0.0706383562469458]    
 [1.1286490332475112, 0.12782139462106018, 0.1372108254749962, 0.109917644216865, 0.05390755413708694, 0.06751060755403648, 0.0815076354724247, 0.047760952859374046, 0.09906618191993528]     
 [1.143206654033543, 0.07803699061990163, 0.09568792229132819, 0.18225269137262864, 0.00899893243447417, 0.014749588453081015, 0.0879192606939057, 0.05369526074068288, 0.10059001828730446]   
 [1.01436925639

In [205]:
norm(u_a[10] - [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

0.30000000000000004

In [208]:
norm([1,0,1]-[0,0,1])

1.0

In [17]:
radius = 0.1:0.1:1.0
Reynolds = 300:50:500

for i in 1:length(tgroup)
    x_array,y_array_num = tgroup[i]
    fig,ax = plt.subplots()
    for j in 1:length(y_array_num)
        R = radius[i]
        Rey = Reynolds[j]
        y_array = y_array_num[j]
        ax.plot(x_array, y_array, label = "Radius@ $R , Re = $Rey")
    end
    ax.legend()
    xlabel(L"timesplit")
    ylabel(L"average lambda")
end

In [15]:
tgroup[1]

(Any[1, 2, 4, 5, 8, 10, 16, 20, 25, 40  …  250, 400, 500, 625, 1000, 1250, 2000, 2500, 5000, 10000], Any[[0.0016096154378397534, 0.0016081976291459608, 0.001585757328704263, 0.0015717913378141288, 0.0014957435873040853, 0.0014189618606106314, 0.0012309378747776612, 0.0011656136399248289, 0.0010259055574474674, 0.0006103037496581332  …  -0.0029003533957697755, -0.003936718658149236, -0.0041706583250232495, -0.004609358255978189, -0.00481638899062621, -0.004801910184676438, -0.004860815468560919, -0.004812588339678061, -0.00524589582178858, -0.006063078915245622], [0.006410399525817924, 0.006404030133404985, 0.006381489993428582, 0.006357090380135984, 0.006284749155123037, 0.00622525098181294, 0.006036863920584003, 0.005960555982586826, 0.0058080642143103715, 0.005325250281328128  …  0.00032208046123531717, -0.0007675653468105936, -0.001205961763158909, -0.001614444235224495, -0.0022039328128850544, -0.002256302183228986, -0.00225736930260934, -0.002228771492010299, -0.002431668932056669